In [1]:
import json
import re
import time
from typing import List

import evaluate
import kscope
from utils import split_prompts_into_batches

# Getting Started

There is a bit of documentation on how to interact with the large models [here](https://kaleidoscope-sdk.readthedocs.io/en/latest/). The relevant github links to the SDK are [here](https://github.com/VectorInstitute/kaleidoscope-sdk) and underlying code [here](https://github.com/VectorInstitute/kaleidoscope).

First we connect to the service through which we'll interact with the LLMs and see which models are avaiable to us

In [2]:
# Establish a client connection to the kscope service
client = kscope.Client(gateway_host="llm.cluster.local", gateway_port=3001)

Show all supported models

In [3]:
client.models

['OPT-175B', 'OPT-6.7B', 'GPT2']

Show all model instances that are currently active

In [4]:
client.model_instances

[{'id': '0a8818f1-991a-48ef-9aa6-0bb4b06b1fe9',
  'name': 'OPT-175B',
  'state': 'ACTIVE'}]

To start, we obtain a handle to a model. In this example, let's use the OPT-175B model.

In [5]:
model = client.load_model("OPT-175B")
# If this model is not actively running, it will get launched in the background.
# In this case, wait until it moves into an "ACTIVE" state before proceeding.
while model.state != "ACTIVE":
    time.sleep(1)

We need to configure the model to generate in the way we want it to. So we set a number of important parameters. For a discussion of the configuration parameters see: `src/reference_implementations/prompting_vector_llms/CONFIG_README.md`

In [6]:
long_generation_config = {"max_tokens": 100, "top_k": 4, "top_p": 1.0, "rep_penalty": 1.2, "temperature": 0.5}

We're going to try out some few shot and zero shot translation from French to English. We take a sample from the very large WMT14 translation dataset, specifically considering French->English translation only

In [7]:
french_texts = []
english_texts = []
with open("resources/translation_dataset/wmt14_sample.json") as file:
    data = json.load(file)["dataset"]
    for french_english_pair in data:
        french_text = french_english_pair["fr"]
        french_texts.append(french_text)
        english_text = french_english_pair["en"]
        english_texts.append(english_text)

### Zero-shot Prompt

For our zero-shot prompt example, we use the same format at the original [GPT-3 paper](https://arxiv.org/pdf/2005.14165.pdf). That is:

Q: What is the {target language} translation of {source text} A: 

In [8]:
zero_shot_prompts = []
for french_text in french_texts:
    zero_shot_prompt = f"Q: What is the English translation of {french_text} A: "
    zero_shot_prompts.append(zero_shot_prompt)

### Few-Shot Prompt

In order to speed up inference a bit, we only use 10-shot prompts for our translation task. The original [GPT-3 paper](https://arxiv.org/pdf/2005.14165.pdf) uses a very large 64-shot prompt to induce their observed performance. The prompt format that we use is distinctly different from the zero-shot setting. We borrow their structure of:

{source text} = {target text}\n\n

but add on an instruction at the beginning 

"Translate the follow sentences from French to English.\n\n"

To create the demonstrations, we take the first `n_examples` from the dataset sample, and use the rest for testing.

In [9]:
n_examples = 10
demonstrations = []
# Create the demonstrations for translation
for french_text, english_text in zip(french_texts[0:n_examples], english_texts[0:n_examples]):
    demonstrations.append(f"{french_text} = {english_text}\n\n")

demonstration_str = "".join(demonstrations)
demonstration_str = f"Translate the follow sentences from French to English.\n\n{demonstration_str}"

Let's give each a try with some basic french sentences

In [10]:
example_1 = "J'aime mon chien."  # I love my dog.
example_2 = "Il y a des gens partout."  # There are people everywhere.
example_3 = (
    "Jusqu'à présent, l'hiver a été étrange à Toronto."  # It has been an very weird winter in Toronto thus far.
)

### Zero Shot Examples.

We only grab the first sentence in the response because we are targeting translation of only one sentence

In [11]:
# Place examples in the zero shot template
zero_shot_prompt = f"Q: What is the English translation of {example_1} A: "
generation = model.generate(zero_shot_prompt, long_generation_config)
# Grab the first sentence output.
print(re.findall(r".*?[.!\?]", generation.generation["text"][0])[0])

 I love my dog.


In [12]:
zero_shot_prompt = f"Q: What is the English translation of {example_2} A: "
generation = model.generate(zero_shot_prompt, long_generation_config)
# Grab the first sentence output.
print(re.findall(r".*?[.!\?]", generation.generation["text"][0])[0])

 There are people everywhere.


In [13]:
zero_shot_prompt = f"Q: What is the English translation of {example_3} A: "
generation = model.generate(zero_shot_prompt, long_generation_config)
# Grab the first sentence output.
print(re.findall(r".*?[.!\?]", generation.generation["text"][0])[0])

 To this point, the winter has been strange in Toronto.


Zero-shot isn't too bad, sometimes get a fairly good translation.

### Few-shot Examples 

Again, we only grab the first sentence in the response because we are targeting translation of only one sentence

In [14]:
# Place examples in the few shot template
few_shot_prompt = f"{demonstration_str}{example_1} = "

In [15]:
generation = model.generate(few_shot_prompt, long_generation_config)
# Grab the first sentence output.
print(re.findall(r".*?[.!\?]", generation.generation["text"][0])[0])

Le chien est bien.


In [16]:
# Place examples in the zero shot template
few_shot_prompt = f"{demonstration_str}{example_2} = "
generation = model.generate(few_shot_prompt, long_generation_config)
# Grab the first sentence output.
print(re.findall(r".*?[.!\?]", generation.generation["text"][0])[0])

ˆThere are people everywhere.


In [17]:
# Place examples in the zero shot template
few_shot_prompt = f"{demonstration_str}{example_3} = "
generation = model.generate(few_shot_prompt, long_generation_config)
# Grab the first sentence output.
print(re.findall(r".*?[.!\?]", generation.generation["text"][0])[0])

Â« Until now, the winter has been strange in Toronto.


### Let's measure the BLEU scores for the dataset that we have sampled.

In [18]:
bleu_metric = evaluate.load("bleu")

### Zero-shot example

In [19]:
# Split prompts into batches for memory management.
translations = []
zero_shot_batches = split_prompts_into_batches(zero_shot_prompts)
for batch_number, zero_shot_batch in enumerate(zero_shot_batches):
    generations = model.generate(zero_shot_batch, long_generation_config)
    print(f"Batch number {batch_number+1} Complete")
    for single_generation in generations.generation["text"]:
        # Take just the first sentence
        generation_text = re.findall(r".*?[.!\?]", single_generation)
        generation_text = generation_text[0] if len(generation_text) > 0 else single_generation
        translations.append(generation_text)

Batch number 1 Complete
Batch number 2 Complete
Batch number 3 Complete
Batch number 4 Complete
Batch number 5 Complete
Batch number 6 Complete
Batch number 7 Complete
Batch number 8 Complete
Batch number 9 Complete
Batch number 10 Complete


In [20]:
def convert_references_for_bleu(references: List[str]) -> List[List[str]]:
    # The bleu metric requires inputs to be stored as lists of lists. So we encapsulate each reference in a list
    return [[reference] for reference in references]

In [21]:
bleu_metric.compute(predictions=translations, references=convert_references_for_bleu(english_texts))

{'bleu': 0.10254306983969617,
 'precisions': [0.3072719630096679,
  0.1448577680525164,
  0.08933454876937101,
  0.05985748218527316],
 'brevity_penalty': 0.8255733409569715,
 'length_ratio': 0.8391534391534392,
 'translation_length': 2379,
 'reference_length': 2835}

### Few-shot example

__NOTE__: This takes quite a while to run due to the sequence length associated with 10-shot prompts.

In [22]:
# Note that we're only taking the remaining 90 examples from the test set, since we used the first 10 for few-shot
# examples
few_shot_prompts = [f"{demonstration_str}{french_text} = " for french_text in french_texts[n_examples:]]

In [23]:
# Split prompts into batches for memory management.
translations = []
few_shot_batches = split_prompts_into_batches(few_shot_prompts)
for batch_number, few_shot_batch in enumerate(few_shot_batches):
    generations = model.generate(few_shot_batch, long_generation_config)
    print(f"Batch number {batch_number+1} Complete")
    for single_generation in generations.generation["text"]:
        # Take just the first sentence
        generation_text = re.findall(r".*?[.!\?]", single_generation)
        generation_text = generation_text[0] if len(generation_text) > 0 else single_generation
        translations.append(generation_text)

Batch number 1 Complete
Batch number 2 Complete
Batch number 3 Complete
Batch number 4 Complete
Batch number 5 Complete
Batch number 6 Complete
Batch number 7 Complete
Batch number 8 Complete
Batch number 9 Complete


In [24]:
# Note that we're only taking the remaining 90 examples from the test set, since we used the first 10 for few-shot
# examples
bleu_metric.compute(predictions=translations, references=convert_references_for_bleu(english_texts[n_examples:]))

{'bleu': 0.28117811753944133,
 'precisions': [0.5707779886148008,
  0.33398821218074654,
  0.21724979658258747,
  0.15092748735244518],
 'brevity_penalty': 1.0,
 'length_ratio': 1.010740314537783,
 'translation_length': 2635,
 'reference_length': 2607}

Clearly the few-shot prompt does a much better job in translation, at least as measured by the BLEU Score